## Домашняя работа 6
фев'24
<hr>

### Часть 1. EDA

<br>Скачайте данные с Kaggle по ценам на жильё в Airbnb в Нью-Йорке:
<br>Пройдите по основным шагам работы с данными:
<ul><li>выкиньте ненужные признаки: id, name, host_id, host_name, last_review</li>
<li>визуализируйте базовые статистики данных: распределения признаков, матрицу попарных корреляций, постройте pair plots</li>
<li>про результатам анализа произведите предобработку переменных</li></ul>

<hr>

### Часть 2. Preprocessing & Feature Engineering

<br>Ваша цель получить как можно более высокие метрики качества (можно взять несколько, R2, MAE, RMSE), сконцентрировавшись на преобразовании признаков.
<br>Опробуйте различные техники:
<ul><li>работа с категориальными переменными (можно начать с dummy);</li>
<li>замена аномалий;</li>
<li>различные варианты шкалирования непрерывных переменных (StandardScaler, RobustScaler, и.т.д.);</li>
<li>обратите внимание на распределение целевой переменной, возможно, с ней тоже можно поработать;</li>
</ul>
Попробуйте на основании имеющихся переменных создать новые, которые могли бы улучшить качество модели. Например, можно найти координаты Манхэттена (самого дорогого района) и при помощи широты и долготы, а также евклидового расстояния создать новую переменную - расстояние от квартиры до этого района. Возможно, такой признак будет работать лучше, чем просто широта и долгота.

<hr>

### Часть 3. Моделирование

<br>Отложите 30% данных для тестирования.
<br>Постройте модели простой линейной регрессии, RidgeCV, LassoCV и ElasticNetCV.
<br>Измерьте качество каждой и визуализируйте важность признаков.
<br>Сделайте интересные выводы :)